<a href="https://colab.research.google.com/github/SeyedMehdi110/Per-1/blob/main/Question7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#importing required libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cvxpy as cp
from sklearn.metrics import accuracy_score 

In [12]:
#reading data
dataframe = pd.read_csv('train.csv',header = None)

dataframe[0].replace(0, -1,inplace = True)
y = dataframe[0]
X = dataframe.drop([0], axis=1)

#splitting data
data_train, data_val, label_train, label_val = train_test_split(X, y, test_size=0.42, random_state=1)

#importing test data
test_set = pd.read_csv('test.csv',header = None)
test_set[0].replace(0, -1,inplace = True)

label_test = test_set[0]
data_test = test_set.drop([0], axis=1)


data_train=np.array(data_train)
data_test=np.array(data_test)
data_val=np.array(data_val)

label_train = np.array(label_train)
label_test = np.array(label_test)
label_val = np.array(label_val)


In [20]:
import numpy as np
import cvxopt


class SupportVectorMachine(object):
    def __init__(self, C=None):
        self.C = C

        
    def fit(self, X, y):
        n = X.shape[0]
          
        # Calculate the Gram matrix.
        K = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                K[i,j] = np.dot(X[i], X[j])
        
        # Setup necessary matrices to be used in the calculation of the Lagrangian
        # multipliers.
        P = cvxopt.matrix(np.outer(y,y) * K)        
        q = cvxopt.matrix(-np.ones((n,1)))
        A = cvxopt.matrix(y.reshape(1,-1))
        b = cvxopt.matrix(np.zeros((1,1)))
        
        if self.C is None:
            h = cvxopt.matrix(np.zeros((n, 1)))
            G = cvxopt.matrix(-np.eye(n))
        else:
            G_lower = -np.eye(n)
            G_upper = np.eye(n)
            h_lower = np.zeros((n,1))
            h_upper = np.full((n,1),self.C)
            G = cvxopt.matrix(np.concatenate((G_lower, G_upper), axis=0))
            h = cvxopt.matrix(np.concatenate((h_lower,h_upper), axis=0))
        
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)  
        
        # Extract the Lagrangian multipliers.
        self.alphas = (np.array(solution['x'])).flatten()
        
        self.support_alphas_indices = []
        self.support_alphas = []
        self.support_vectors = []
        self.support_ys = []
        
        # Determine the support vectors, their associated Lagrangian multipliers and labels.
        for i, x in enumerate(self.alphas):
            if x > 1e-4:
                self.support_alphas_indices.append(i)
                self.support_alphas.append(self.alphas[i])
                self.support_vectors.append(X[i])
                self.support_ys.append(y[i])


        print("Sum of Alphas= ",np.sum(self.alphas))  
        self.support_alphas_indices = np.array(self.support_alphas_indices)
        self.support_alphas = np.array(self.support_alphas)
        self.support_vectors = np.array(self.support_vectors)
        self.support_ys = np.array(self.support_ys)
        
        isSV = self.alphas > 1e-5
        
        # Calculate the model intercept.
        self.b = 0
        for idx, i in enumerate(self.support_alphas_indices):
            self.b += self.support_ys[idx]
            kernel_vec = K[i]
            for j,k,l in zip(self.support_alphas, self.support_ys, kernel_vec[isSV]):
                self.b -= (j * k * l)
        if self.support_alphas.any()!=0:
          self.b /= len(self.support_alphas)


        
        
        # Calculate the model weights if applicable.
        self.w = np.zeros(X.shape[1])
        for i in range(len(self.support_alphas)):
            multiplier = self.support_alphas[i] * self.support_ys[i]
            for x in range(X.shape[1]):
                self.w[x] +=  multiplier * self.support_vectors[i][x]

            
            
    def decision_function(self,X):
        '''Calculate and return the real valued prediction of the model.'''
        try:
            if self.w is not None:
                return X @ self.w + self.b
            else:
                preds = []
                for x in X:
                    summation = 0
                    for alpha,y,sv in zip(self.support_alphas, self.support_ys, self.support_vectors):
                        summation += alpha * y * self.apply_kernel_function(x,sv)
                    preds.append(summation)
                return preds + self.b   
        except AttributeError:
            print('Model not fitted, call \'fit\' with appropriate arguments before using model.')
            return -1
            
            
    def predict(self,X):
        '''Return the models predicted class for each of the given instances.'''
        try:
            return np.sign(self.decision_function(X))
        except AttributeError:
            print('Model not fitted, call \'fit\' with appropriate arguments before using model.')

In [32]:
Cs=[2**-10,2**-8,2**-6,2**-4,2**-2,1,2,4,16,64,256,1024]

for C in Cs:
  svm_train_dual=SupportVectorMachine(C=C)
  svm_train_dual.fit(data_train , label_train)
  def sign(x):
    if x < 0:
        return -1
    return 1
  #prediction and calculating accuracy function
  pred = svm_train_dual.predict(data_test)
  prediction = []
  for i in pred:
      prediction.append(sign(i))
  prediction = np.array(prediction)
  acc = accuracy_score(label_test,prediction)
  print("accuracy_score for C= ", C," is ",acc)



     pcost       dcost       gap    pres   dres
 0: -4.6740e+02 -1.1183e+01  5e+04  2e+02  6e-13
 1: -5.5893e+00 -1.1175e+01  5e+02  2e+00  6e-13
 2: -9.3610e-01 -1.0485e+01  3e+01  1e-01  3e-14
 3: -5.2739e-01 -5.3593e+00  7e+00  1e-02  4e-15
 4: -4.2592e-01 -1.1998e+00  9e-01  1e-03  5e-15
 5: -5.2183e-01 -8.5778e-01  4e-01  5e-04  3e-15
 6: -5.6765e-01 -7.4094e-01  2e-01  2e-04  2e-15
 7: -5.8431e-01 -7.0190e-01  1e-01  1e-04  2e-15
 8: -6.0161e-01 -6.6294e-01  6e-02  5e-05  2e-15
 9: -6.1407e-01 -6.4067e-01  3e-02  2e-05  2e-15
10: -6.2044e-01 -6.3012e-01  1e-02  4e-06  2e-15
11: -6.2346e-01 -6.2563e-01  2e-03  4e-07  2e-15
12: -6.2435e-01 -6.2452e-01  2e-04  1e-08  3e-15
13: -6.2443e-01 -6.2443e-01  5e-06  4e-10  3e-15
14: -6.2443e-01 -6.2443e-01  9e-08  6e-12  3e-15
Optimal solution found.
Sum of Alphas=  0.83610694133055
accuracy_score for C=  0.0009765625  is  0.8766666666666667
     pcost       dcost       gap    pres   dres
 0: -4.6877e+02 -4.4603e+01  5e+04  2e+02  6e-13
 1: